In [7]:
from functools import partial, reduce
from os import rename, makedirs
import os
import urllib.request
import pandas as pd
import os.path
import shutil
import json
import csv
import re
import datetime
from typing import Literal

In [8]:
# carpeta = f"../output/AGROPE1/definitivos"

# csv_files = [x for x in os.listdir(carpeta) if ".csv" in x]

# for f in csv_files: 
#     new = f"{carpeta}/{f.replace('_old','')}"
#     old = f"{carpeta}/{f}"
#     os.rename(old, new)


In [9]:
def gsheet_download_csv(id: str, target: str, url_template="https://docs.google.com/spreadsheets/d/{id}/export?format=csv"):
    url = url_template.format(id=id)
    return urllib.request.urlretrieve(url, target)

gsheet_download_csv('1UyfAnRCes0OAOUR0aARfkfegTsUl4Puw', 'nomenclador.csv')
paises = pd.read_csv('./nomenclador.csv')
pais = {k:v for k,v in paises[['codigo_fundar', 'desc_fundar']].iloc}
codes_set = set(pais.keys())

lista_cambios = []
diccionario_cambios = {}

In [10]:
def match_relocate(src, target, pattern, file_list=None):
    files_to_move = file_list or os.listdir(src)

    pattern = re.compile(pattern)

    if not os.path.exists(target):
        makedirs(target)

    for filename in files_to_move:
        if pattern.match(filename):
            src_path = os.path.join(src, filename)
            dest_path = os.path.join(target, filename)

            shutil.move(src_path, dest_path)

    print(f"Files matching the pattern '{pattern}' moved to '{target}'.")

def compose2(f, g):
    def composed(*a, **kw):
        return f(g(*a, **kw))

    return composed

def compose(*fs):
    return reduce(compose2, fs)

def _chain(*fs):
    return compose(*reversed(fs))

def chain(*fs):
    lista_cambios.clear()
    return _chain(*fs)

def exportar_definitivo(archivo: str, folder:str, df: pd.DataFrame, nuevo_nombre=None):
    if not nuevo_nombre:
        nuevo_nombre = f"{folder}/definitivos/{archivo}_old.csv"

    original_file = f'{folder}/definitivos/{archivo}.csv'

    rename(original_file, nuevo_nombre)

    df.to_csv(original_file, 
              encoding='utf-8', 
              sep=',', 
              quoting=csv.QUOTE_ALL, 
              quotechar='"', 
              lineterminator='\n', 
              decimal='.', 
              index=False)
    
    return original_file, nuevo_nombre

def drop_col(df: pd.DataFrame, col, axis=1, lista_cambios = lista_cambios):
    lista_cambios.append(f"Se borró la columna {col}")
    return df.drop(col, axis=axis)

def drop_colx(col, axis=1):
    return lambda df: drop_col(df, col, axis=axis)

def normalize_countries(df: pd.DataFrame, code_col:str, new_col:str, desc_col:str, map: dict, lista_cambios = lista_cambios):
    codes_set = set(map.keys())
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    diff_set = set(df[code_col].dropna()) - codes_set
    if len(diff_set)>0: 
        diff_str = ", ".join([str(x) for x in list(diff_set)])
        string = f"ERROR: Los códigos {diff_str} no se encontraron en el nomenclador vigente al {today} en la columna {code_col}"
        lista_cambios.append(string)
    else:
        if (new_col and desc_col):
            df[new_col] = df[code_col].apply(map.__getitem__)
            df = df.drop(desc_col, axis=1)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, creando la columna {new_col} y borrando la columna {desc_col}"
        elif desc_col:
            df[desc_col] = df[code_col].apply(map.__getitem__)
            string = f"Usando los valores de la columna {code_col}, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al {today}, cambiando los valores de la columna {desc_col}"
        else: 
            string = f"Usando los valores de la columna {code_col}, se verificó que todos los códigos se encuentran en el nomenclador vigente al {today}"
        lista_cambios.append(string)
    return df

def normalize_countriesx(code_col, new_col, desc_col, map):
    return lambda df: normalize_countries(df, code_col=code_col, new_col=new_col, desc_col=desc_col, map=map)

def wide_to_long(df: pd.DataFrame, primary_keys, value_name='valor', var_name='indicador', lista_cambios=lista_cambios):
    cols = ", ".join(df.columns)
    new_cols = ", ".join(primary_keys + [value_name, var_name])
    string = f"Long estricto: [{cols}] -> [{new_cols}]"
    lista_cambios.append(string)
    return df.melt(id_vars=primary_keys, value_name=value_name, var_name=var_name)

def wide_to_longx(primary_keys, value_name='valor', var_name='indicador'):
    return lambda df: wide_to_long(df, primary_keys, value_name=value_name, var_name=var_name)

def rename_cols(df: pd.DataFrame, map, lista_cambios=lista_cambios):
    map_string = ", ".join(f"{old_col}:{new_col}" for old_col, new_col in map.items())
    string = f"Se renombraron las columnas: {map_string}"
    lista_cambios.append(string)
    df = df.rename(columns=map)
    return df

def rename_colsx(map):
    return lambda df: rename_cols(df, map)

def replace_value(df:pd.DataFrame, col:str, curr_value:str, new_value:str, lista_cambios=lista_cambios):
    string = f"Se imputó el valor {new_value} en la columna {col} cuando {col}=={curr_value}"
    lista_cambios.append(string)
    df = df.replace({col: curr_value}, new_value)
    return df

def replace_valuex(col, curr_value, new_value):
    return lambda df: replace_value(df=df, col=col, curr_value=curr_value, new_value=new_value)

def sort_vals_asc(df:pd.DataFrame, prim_keys:list, lista_cambios=lista_cambios):
    lista_cambios.append(f"Ordené las filas por las columnas: {', '.join(prim_keys)}")
    return df.sort_values(by=prim_keys).reset_index(drop=True)

def sort_vals_ascx(prim_keys):
    return lambda df: sort_vals_asc(df=df, prim_keys=prim_keys)

def cast_col(df:pd.DataFrame, col:str, cast_to:Literal['int','float','object'], lista_cambios=lista_cambios):
    old_type = str(df[col].dtype)
    df[col] = df[col].astype(cast_to)
    lista_cambios.append(f"El tipo de dato de la columna {col} fue cambiado {old_type} ->> {cast_to}")
    return df

def cast_colx(col, cast_to):
    return lambda df: cast_col(df=df, col=col, cast_to=cast_to)

AGROPE_g1.csv

In [11]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 1

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,participacion_agro
0,1935,0.21
1,1936,0.21
2,1937,0.23
3,1938,0.20
4,1939,0.19


In [12]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)


pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)


diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1935,participacion_agro,0.21
1,1936,participacion_agro,0.21
2,1937,participacion_agro,0.23
3,1938,participacion_agro,0.20
4,1939,participacion_agro,0.19


In [13]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [14]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g1.csv',
 '../output/AGROPE1/definitivos/AGROPE_g1_old.csv')

AGROPE_g2.csv

In [15]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,pbi_agro_pcons
0,1875,971
1,1876,1139
2,1877,1302
3,1878,1237
4,1879,1330


In [16]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1875,pbi_agro_pcons,971
1,1876,pbi_agro_pcons,1139
2,1877,pbi_agro_pcons,1302
3,1878,pbi_agro_pcons,1237
4,1879,pbi_agro_pcons,1330


In [17]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio']}

In [18]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g2.csv',
 '../output/AGROPE1/definitivos/AGROPE_g2_old.csv')

AGROPE_g3.csv

In [19]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 3

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio cuenta


,anio,cuenta,participacion_pbi
0,2004,Agricultura,0.05
1,2005,Agricultura,0.04
2,2006,Agricultura,0.04
3,2007,Agricultura,0.05
4,2008,Agricultura,0.05


In [20]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    to_long(primary_keys),
    sort_vals_ascx(primary_keys)
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'region_code': 'cod_area'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cuenta,indicador,valor
0,2004,Agricultura,participacion_pbi,0.050
1,2004,Otros,participacion_pbi,0.010
2,2004,Pecuario,participacion_pbi,0.020
3,2004,Total,participacion_pbi,0.081
4,2005,Agricultura,participacion_pbi,0.040


In [21]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta']}

In [22]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g3.csv',
 '../output/AGROPE1/definitivos/AGROPE_g3_old.csv')

AGROPE_g4.csv

In [23]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 4

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3c anio


,iso3c,anio,va_agro_sobre_pbi,pais
0,HIC,2022,NaN,Países de ingreso alto
1,HIC,2021,1.276669,Países de ingreso alto
2,HIC,2020,1.285265,Países de ingreso alto
3,HIC,2019,1.217419,Países de ingreso alto
4,HIC,2018,1.267516,Países de ingreso alto


In [24]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3c,anio,pais,indicador,valor
0,ARG,1960,Argentina,va_agro_sobre_pbi,NaN
1,ARG,1961,Argentina,va_agro_sobre_pbi,NaN
2,ARG,1962,Argentina,va_agro_sobre_pbi,NaN
3,ARG,1963,Argentina,va_agro_sobre_pbi,NaN
4,ARG,1964,Argentina,va_agro_sobre_pbi,NaN


In [25]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio']}

In [26]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g4.csv',
 '../output/AGROPE1/definitivos/AGROPE_g4_old.csv')

AGROPE_g5.csv

In [27]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 5

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3c anio


,iso3c,pais,anio,va_agro_sobre_pbi
0,MIC,Ingreso mediano,2021,8.890954
1,WLD,Media mundial,2021,4.304093
2,ARG,Argentina,2021,7.131541
3,BOL,Bolivia,2021,12.922253
4,BRA,Brasil,2021,7.486921


In [28]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3c,anio,pais,indicador,valor
0,ARG,2021,Argentina,va_agro_sobre_pbi,7.131541
1,BOL,2021,Bolivia,va_agro_sobre_pbi,12.922253
2,BRA,2021,Brasil,va_agro_sobre_pbi,7.486921
3,CHL,2021,Chile,va_agro_sobre_pbi,3.645349
4,COL,2021,Colombia,va_agro_sobre_pbi,7.635340


In [29]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [30]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g5.csv',
 '../output/AGROPE1/definitivos/AGROPE_g5_old.csv')

AGROPE_g6.csv

In [31]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 6

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio cod_pcia


,anio,nom_pcia,valor,cod_pcia
0,2004,Ciudad de Buenos Aires,0.0021,2
1,2005,Ciudad de Buenos Aires,0.0020,2
2,2006,Ciudad de Buenos Aires,0.0017,2
3,2007,Ciudad de Buenos Aires,0.0018,2
4,2008,Ciudad de Buenos Aires,0.0018,2


In [32]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['nom_pcia'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,cod_pcia,nom_pcia,indicador,valor
0,2004,2,Ciudad de Buenos Aires,valor,0.0021
1,2004,6,Buenos Aires,valor,0.0889
2,2004,10,Catamarca,valor,0.0501
3,2004,14,Córdoba,valor,0.2114
4,2004,18,Corrientes,valor,0.0660


In [33]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [34]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g6.csv',
 '../output/AGROPE1/definitivos/AGROPE_g6_old.csv')

AGROPE_g7.csv

In [35]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 7

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,q_total,share
0,Algodón,2021/22,1115510,0.777280
1,Alpiste,2021/22,36114,0.025164
2,Arroz,2021/22,1222426,0.851779
3,Arveja,2021/22,296957,0.206918
4,Avena,2021/22,723154,0.503889


In [36]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Algodón,2021/22,q_total,1.115510e+06
1,Algodón,2021/22,share,7.772804e-01
2,Alpiste,2021/22,q_total,3.611400e+04
3,Alpiste,2021/22,share,2.516401e-02
4,Arroz,2021/22,q_total,1.222426e+06


In [37]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [38]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g7.csv',
 '../output/AGROPE1/definitivos/AGROPE_g7_old.csv')

AGROPE_g8.csv

In [39]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 8

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,q_total
0,Maíz,1969/70,9358372.0
1,Maíz,1970/71,9928194.0
2,Maíz,1971/72,5858318.0
3,Maíz,1972/73,9698405.0
4,Maíz,1973/74,9898570.0


In [40]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,q_total,9358372.0
1,Maíz,1970/71,q_total,9928194.0
2,Maíz,1971/72,q_total,5858318.0
3,Maíz,1972/73,q_total,9698405.0
4,Maíz,1973/74,q_total,9898570.0


In [41]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [42]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g8.csv',
 '../output/AGROPE1/definitivos/AGROPE_g8_old.csv')

AGROPE_g9.csv

In [43]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 9

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,supsem_total
0,Maíz,1969/70,4663760.0
1,Maíz,1970/71,4990990.0
2,Maíz,1971/72,4437630.0
3,Maíz,1972/73,4249455.0
4,Maíz,1973/74,4132500.0


In [44]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,supsem_total,4663760.0
1,Maíz,1970/71,supsem_total,4990990.0
2,Maíz,1971/72,supsem_total,4437630.0
3,Maíz,1972/73,supsem_total,4249455.0
4,Maíz,1973/74,supsem_total,4132500.0


In [45]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [46]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g9.csv',
 '../output/AGROPE1/definitivos/AGROPE_g9_old.csv')

AGROPE_g10.csv

In [47]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 10

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cultivo campania


,cultivo,campania,rindes_avg
0,Maíz,1969/70,2330.502874
1,Maíz,1970/71,2442.963527
2,Maíz,1971/72,1862.165290
3,Maíz,1972/73,2720.995936
4,Maíz,1973/74,2840.376476


In [48]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3c', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cultivo,campania,indicador,valor
0,Maíz,1969/70,rindes_avg,2330.502874
1,Maíz,1970/71,rindes_avg,2442.963527
2,Maíz,1971/72,rindes_avg,1862.165290
3,Maíz,1972/73,rindes_avg,2720.995936
4,Maíz,1973/74,rindes_avg,2840.376476


In [49]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [50]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g10.csv',
 '../output/AGROPE1/definitivos/AGROPE_g10_old.csv')

AGROPE_g11.csv

In [51]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 11

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,rindes_maiz,pais,rindes_maiz_ma5
0,ARG,1961,1.7672,argentina,NaN
1,ARG,1962,1.8936,argentina,NaN
2,ARG,1963,1.6481,argentina,NaN
3,ARG,1964,1.8010,argentina,NaN
4,ARG,1965,1.6785,argentina,1.75768


In [52]:
# m49_cod = paises[['m49_code_unsd', 'iso3_desc_fundar']].dropna(how="any").sort_values(by="m49_code_unsd").reset_index(drop=True)
# m49_cod['m49_code_unsd'] = m49_cod['m49_code_unsd'].astype('int')
# pais_m49_cod = {k:v for k,v in m49_cod.iloc}
# codes_set_m49_cod = set(pais_m49_cod.keys())


In [53]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais,indicador,valor
0,ARG,1961,Argentina,rindes_maiz,1.7672
1,ARG,1961,Argentina,rindes_maiz_ma5,NaN
2,ARG,1962,Argentina,rindes_maiz,1.8936
3,ARG,1962,Argentina,rindes_maiz_ma5,NaN
4,ARG,1963,Argentina,rindes_maiz,1.6481


In [54]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [55]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g11.csv',
 '../output/AGROPE1/definitivos/AGROPE_g11_old.csv')

AGROPE_g12.csv

In [56]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 12

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,rindes_soja,pais,rindes_soja_ma5
0,ARG,1961,0.9765,argentina,NaN
1,ARG,1962,1.1628,argentina,NaN
2,ARG,1963,0.9802,argentina,NaN
3,ARG,1964,1.1457,argentina,NaN
4,ARG,1965,1.0352,argentina,1.06008


In [57]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais,indicador,valor
0,ARG,1961,Argentina,rindes_soja,0.9765
1,ARG,1961,Argentina,rindes_soja_ma5,NaN
2,ARG,1962,Argentina,rindes_soja,1.1628
3,ARG,1962,Argentina,rindes_soja_ma5,NaN
4,ARG,1963,Argentina,rindes_soja,0.9802


In [58]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [59]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g12.csv',
 '../output/AGROPE1/definitivos/AGROPE_g12_old.csv')

AGROPE_g13.csv

In [60]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 13

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,anio,rindes_trigo,pais,rindes_trigo_ma5
0,ARG,1961,1.2950,argentina,NaN
1,ARG,1962,1.5222,argentina,NaN
2,ARG,1963,1.5751,argentina,NaN
3,ARG,1964,1.8353,argentina,NaN
4,ARG,1965,1.3212,argentina,1.50976


In [61]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais']),
    sort(primary_keys),
    # renombrar({'iso3': 'cod_fundar','time':'anio'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,pais,indicador,valor
0,ARG,1961,Argentina,rindes_trigo,1.2950
1,ARG,1961,Argentina,rindes_trigo_ma5,NaN
2,ARG,1962,Argentina,rindes_trigo,1.5222
3,ARG,1962,Argentina,rindes_trigo_ma5,NaN
4,ARG,1963,Argentina,rindes_trigo,1.5751


In [62]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [63]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g13.csv',
 '../output/AGROPE1/definitivos/AGROPE_g13_old.csv')

AGROPE_g14.csv

In [64]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 14

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,iso3_desc_fundar,anio,valor
0,AFG,Afganistán,1961,7750.0
1,AFG,Afganistán,1962,7800.0
2,AFG,Afganistán,1963,7850.0
3,AFG,Afganistán,1964,7905.0
4,AFG,Afganistán,1965,7910.0


In [65]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,iso3_desc_fundar,indicador,valor
0,ABW,1961,Aruba,valor,2.0
1,ABW,1962,Aruba,valor,2.0
2,ABW,1963,Aruba,valor,2.0
3,ABW,1964,Aruba,valor,2.0
4,ABW,1965,Aruba,valor,2.0


In [66]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [67]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g14.csv',
 '../output/AGROPE1/definitivos/AGROPE_g14_old.csv')

AGROPE_g15.csv

In [68]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 15

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,iso3_desc_fundar,anio,valor
0,ALB,Albania,1980,6000.0
1,ALB,Albania,1981,6500.0
2,ALB,Albania,1982,7500.0
3,ALB,Albania,1983,7000.0
4,ALB,Albania,1984,7500.0


In [69]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,iso3_desc_fundar,indicador,valor
0,AGO,1991,Angola,valor,0.0
1,AGO,1992,Angola,valor,0.0
2,AGO,1993,Angola,valor,0.0
3,AGO,1994,Angola,valor,0.0
4,AGO,1995,Angola,valor,0.0


In [70]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [71]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g15.csv',
 '../output/AGROPE1/definitivos/AGROPE_g15_old.csv')

AGROPE_g16.csv

In [72]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 16

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,iso3,iso3_desc_fundar,anio,valor
0,AFG,Afganistán,1961,700000.0
1,AFG,Afganistán,1962,700000.0
2,AFG,Afganistán,1963,713000.0
3,AFG,Afganistán,1964,720000.0
4,AFG,Afganistán,1965,720000.0


In [73]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,iso3_desc_fundar,indicador,valor
0,AFG,1961,Afganistán,valor,700000.0
1,AFG,1962,Afganistán,valor,700000.0
2,AFG,1963,Afganistán,valor,713000.0
3,AFG,1964,Afganistán,valor,720000.0
4,AFG,1965,Afganistán,valor,720000.0


In [74]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [75]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g16.csv',
 '../output/AGROPE1/definitivos/AGROPE_g16_old.csv')

AGROPE_g17.csv

In [76]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 17

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio tipo_carne


,tipo_carne,anio,valor
0,Carne bovina,1961,2145064.0
1,Carne bovina,1962,2378826.0
2,Carne bovina,1963,2605287.0
3,Carne bovina,1964,2019240.0
4,Carne bovina,1965,1995096.0


In [77]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,tipo_carne,indicador,valor
0,1961,Carne aviar,valor,36241.0
1,1961,Carne bovina,valor,2145064.0
2,1961,Carne de cerdo,valor,186932.0
3,1962,Carne aviar,valor,43474.0
4,1962,Carne bovina,valor,2378826.0


In [78]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [79]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g17.csv',
 '../output/AGROPE1/definitivos/AGROPE_g17_old.csv')

AGROPE_g18.csv

In [80]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 18

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio grupo_carne


,anio,grupo_carne,valor
0,1961,aviar,2.077591
1,1962,aviar,2.421226
2,1963,aviar,2.895741
3,1964,aviar,3.925668
4,1965,aviar,5.792445


In [81]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,grupo_carne,indicador,valor
0,1961,aviar,valor,2.077591
1,1961,caprina_ovina,valor,7.033406
2,1961,otras_carnes,valor,0.912113
3,1961,pescado_mariscos,valor,4.226124
4,1961,porcina,valor,8.806959


In [82]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [83]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g18.csv',
 '../output/AGROPE1/definitivos/AGROPE_g18_old.csv')

AGROPE_g12.csv

In [84]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 19

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 tipo_carne


,iso3,iso3_desc_fundar,tipo_carne,valor
0,AFG,Afganistán,pescado_mariscos,0.359595
1,AFG,Afganistán,otras_carnes,0.299662
2,AFG,Afganistán,aviar,1.708074
3,AFG,Afganistán,caprina_ovina,4.035451
4,AFG,Afganistán,porcina,0.000000


In [85]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor': 'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,tipo_carne,iso3_desc_fundar,indicador,valor
0,AFG,aviar,Afganistán,valor,1.708074
1,AFG,caprina_ovina,Afganistán,valor,4.035451
2,AFG,otras_carnes,Afganistán,valor,0.299662
3,AFG,pescado_mariscos,Afganistán,valor,0.359595
4,AFG,porcina,Afganistán,valor,0.000000


In [86]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [87]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g19.csv',
 '../output/AGROPE1/definitivos/AGROPE_g19_old.csv')

AGROPE_g20.csv

In [88]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 20

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 iso3_desc_fundar


,iso3,iso3_desc_fundar,valor
0,AFG,Afganistán,115281.85
1,ALB,Albania,31712.00
2,DZA,Argelia,146270.00
3,AGO,Angola,108789.55
4,ATG,Antigua y Barbuda,57.00


In [89]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,iso3_desc_fundar,indicador,valor
0,AFG,Afganistán,valor,115281.85
1,AGO,Angola,valor,108789.55
2,ALB,Albania,valor,31712.00
3,ARE,Emiratos Árabes Unidos,valor,19208.21
4,ARG,Argentina,valor,2981690.00


In [90]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [91]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g20.csv',
 '../output/AGROPE1/definitivos/AGROPE_g20_old.csv')

AGROPE_g21.csv

In [92]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 21

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 iso3_desc_fundar


,iso3,iso3_desc_fundar,valor
0,AFG,Afganistán,28029.23
1,ALB,Albania,13250.00
2,DZA,Argelia,257644.63
3,AGO,Angola,39950.63
4,ATG,Antigua y Barbuda,33.60


In [93]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,iso3_desc_fundar,indicador,valor
0,AFG,Afganistán,valor,28029.23
1,AGO,Angola,valor,39950.63
2,ALB,Albania,valor,13250.00
3,ARE,Emiratos Árabes Unidos,valor,56000.00
4,ARG,Argentina,valor,2294101.41


In [94]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [95]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g21.csv',
 '../output/AGROPE1/definitivos/AGROPE_g21_old.csv')

AGROPE_g22.csv

In [96]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 22

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3 anio


,anio,iso3,iso3_desc_fundar,valor
0,1961,AFG,Afganistán,43000.0
1,1962,AFG,Afganistán,45800.0
2,1963,AFG,Afganistán,47250.0
3,1964,AFG,Afganistán,48000.0
4,1965,AFG,Afganistán,48700.0


In [97]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,anio,iso3_desc_fundar,indicador,valor
0,AFG,1961,Afganistán,valor,43000.0
1,AFG,1962,Afganistán,valor,45800.0
2,AFG,1963,Afganistán,valor,47250.0
3,AFG,1964,Afganistán,valor,48000.0
4,AFG,1965,Afganistán,valor,48700.0


In [98]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [99]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g22.csv',
 '../output/AGROPE1/definitivos/AGROPE_g22_old.csv')

AGROPE_g23.csv

In [100]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 23

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio iso3 pais


,anio,share_expo,iso3,pais
0,1962,0.2181,ARG,Argentina
1,1963,0.2567,ARG,Argentina
2,1964,0.2400,ARG,Argentina
3,1965,0.2149,ARG,Argentina
4,1966,0.2268,ARG,Argentina


In [101]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,iso3,pais,indicador,valor
0,1962,ARG,Argentina,share_expo,0.2181
1,1962,AUS,Australia,share_expo,0.2237
2,1962,BRA,Brasil,share_expo,0.0083
3,1962,USA,Estados Unidos,share_expo,0.0099
4,1963,ARG,Argentina,share_expo,0.2567


In [102]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [103]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g23.csv',
 '../output/AGROPE1/definitivos/AGROPE_g23_old.csv')

AGROPE_g24.csv

In [104]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 24

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

year iso3


,year,iso3,pais,share_expo,product
0,1962,ARG,Argentina,0.110198,"Carne bovina, fresca, enfriada o congelada"
1,1963,ARG,Argentina,0.145093,"Carne bovina, fresca, enfriada o congelada"
2,1964,ARG,Argentina,0.150733,"Carne bovina, fresca, enfriada o congelada"
3,1965,ARG,Argentina,0.131679,"Carne bovina, fresca, enfriada o congelada"
4,1966,ARG,Argentina,0.141381,"Carne bovina, fresca, enfriada o congelada"


In [105]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='pais'),
    # drop(col="countryname"),
    to_long(primary_keys + ['pais','product']),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,year,iso3,pais,product,indicador,valor
0,1962,ARG,Argentina,"Carne bovina, fresca, enfriada o congelada",share_expo,0.110198
1,1963,ARG,Argentina,"Carne bovina, fresca, enfriada o congelada",share_expo,0.145093
2,1964,ARG,Argentina,"Carne bovina, fresca, enfriada o congelada",share_expo,0.150733
3,1965,ARG,Argentina,"Carne bovina, fresca, enfriada o congelada",share_expo,0.131679
4,1966,ARG,Argentina,"Carne bovina, fresca, enfriada o congelada",share_expo,0.141381


In [106]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [107]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g24.csv',
 '../output/AGROPE1/definitivos/AGROPE_g24_old.csv')

AGROPE_g25.csv

In [108]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 25

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio region


,anio,region,valor
0,2007,Cuyo,2013334.0
1,2007,NEA,10049067.0
2,2007,NOA,2536581.0
3,2007,Pampeana,42984036.0
4,2007,Patagonia,1194886.0


In [109]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,region,indicador,valor
0,2007,Cuyo,valor,2013334.0
1,2007,NEA,valor,10049067.0
2,2007,NOA,valor,2536581.0
3,2007,Pampeana,valor,42984036.0
4,2007,Patagonia,valor,1194886.0


In [110]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [111]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g25.csv',
 '../output/AGROPE1/definitivos/AGROPE_g25_old.csv')

AGROPE_g26.csv

In [112]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 26

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,valor,nota
0,1935,0.11,valor oficial
1,1936,0.11,valor oficial
2,1937,0.09,valor oficial
3,1938,0.09,valor oficial
4,1939,0.09,valor oficial


In [113]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['nota'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,nota,indicador,valor
0,1935,valor oficial,valor,0.11
1,1936,valor oficial,valor,0.11
2,1937,valor oficial,valor,0.09
3,1938,valor oficial,valor,0.09
4,1939,valor oficial,valor,0.09


In [114]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [115]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g26.csv',
 '../output/AGROPE1/definitivos/AGROPE_g26_old.csv')

AGROPE_g27.csv

In [116]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 27

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio grupo_especie


,anio,grupo_especie,valor
0,1989,crustaceos,11782.6
1,1990,crustaceos,10116.2
2,1991,crustaceos,8990.9
3,1992,crustaceos,25077.2
4,1993,crustaceos,20088.4


In [117]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,grupo_especie,indicador,valor
0,1989,crustaceos,valor,11782.60
1,1989,moluscos,valor,26587.30
2,1989,peces,valor,464940.62
3,1990,crustaceos,valor,10116.20
4,1990,moluscos,valor,34817.50


In [118]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [119]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g27.csv',
 '../output/AGROPE1/definitivos/AGROPE_g27_old.csv')

AGROPE_g28.csv

In [120]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 28

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3


,iso3,iso3_desc_fundar,pescado_mariscos
0,HIC,Países de ingreso alto,26.934196
1,LIC,Países de ingreso bajo,5.259969
2,LMC,Países de ingreso medio bajo,14.936611
3,AFG,Afganistán,0.359595
4,AGO,Angola,13.902185


In [121]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys + ['iso3_desc_fundar']),
    sort(primary_keys),
    #renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,iso3_desc_fundar,indicador,valor
0,AFG,Afganistán,pescado_mariscos,0.359595
1,AGO,Angola,pescado_mariscos,13.902185
2,ALB,Albania,pescado_mariscos,8.683039
3,ARE,Emiratos Árabes Unidos,pescado_mariscos,27.017523
4,ARG,Argentina,pescado_mariscos,6.804048


In [122]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [123]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g28.csv',
 '../output/AGROPE1/definitivos/AGROPE_g28_old.csv')

AGROPE_g29.csv

In [124]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 29

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,valor
0,1993,709.0
1,1994,727.0
2,1995,917.0
3,1996,1011.0
4,1997,1035.0


In [125]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    to_long(primary_keys, value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1993,valor,709.0
1,1994,valor,727.0
2,1995,valor,917.0
3,1996,valor,1011.0
4,1997,valor,1035.0


In [126]:
diccionario_cambios

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [127]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g29.csv',
 '../output/AGROPE1/definitivos/AGROPE_g29_old.csv')

AGROPE_g30.csv

In [128]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 30

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,valor
0,1950.0,0.0
1,1951.0,0.0
2,1952.0,0.0
3,1953.0,0.0
4,1954.0,0.0


In [129]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,1950,valor,0.0
1,1951,valor,0.0
2,1952,valor,0.0
3,1953,valor,0.0
4,1954,valor,0.0


In [130]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [131]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g30.csv',
 '../output/AGROPE1/definitivos/AGROPE_g30_old.csv')

AGROPE_g31.csv

In [132]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 31

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

iso3


,iso3,iso3_desc_fundar,valor
0,ABW,Aruba,1.500000
1,AFG,Afganistán,11107.000000
2,AGO,Angola,2808.000000
3,ALB,Albania,8641.000000
4,ARE,Emiratos Árabes Unidos,2662.941999


In [133]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys + ['iso3_desc_fundar'], value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,iso3,iso3_desc_fundar,indicador,valor
0,ABW,Aruba,valor,1.500000
1,AFG,Afganistán,valor,11107.000000
2,AGO,Angola,valor,2808.000000
3,ALB,Albania,valor,8641.000000
4,ARE,Emiratos Árabes Unidos,valor,2662.941999


In [134]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [135]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g31.csv',
 '../output/AGROPE1/definitivos/AGROPE_g31_old.csv')

AGROPE_g32.csv

In [136]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 32

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,valor
0,2004,0.149
1,2005,0.134
2,2006,0.119
3,2007,0.124
4,2008,0.125


In [137]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,2004,valor,0.149
1,2005,valor,0.134
2,2006,valor,0.119
3,2007,valor,0.124
4,2008,valor,0.125


In [138]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [139]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g32.csv',
 '../output/AGROPE1/definitivos/AGROPE_g32_old.csv')

AGROPE_g33.csv

In [140]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 33

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cadena


,cadena,valor
0,Soja,0.236
1,Maiz,0.137
2,Bovino,0.128
3,Lacteo,0.084
4,Trigo,0.071


In [141]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cadena,indicador,valor
0,Ajo,valor,0.006
1,Algodón,valor,0.003
2,Arroz,valor,0.018
3,Avicola,valor,0.050
4,Berries,valor,0.032


In [142]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [143]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g33.csv',
 '../output/AGROPE1/definitivos/AGROPE_g33_old.csv')

AGROPE_g34.csv

In [144]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 34

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

cadena anio


,cadena,anio,valor
0,Ajo,2001,2.454930e+08
1,Algodón,2001,4.388128e+08
2,Arroz,2001,1.925759e+09
3,Avicola,2001,2.676069e+09
4,Berries,2001,1.023989e+09


In [145]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,cadena,anio,indicador,valor
0,Ajo,2001,valor,2.454930e+08
1,Ajo,2002,valor,2.260352e+08
2,Ajo,2003,valor,2.566837e+08
3,Ajo,2004,valor,2.584419e+08
4,Ajo,2005,valor,1.878109e+08


In [146]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [147]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g34.csv',
 '../output/AGROPE1/definitivos/AGROPE_g34_old.csv')

AGROPE_g35.csv (no se hizo nada porque era un geojson)

In [148]:
diccionario_cambios['AGROPE_g35'] = ["No se hizo ninguna modificación por ser archivo GeoJSON"]

AGROPE_g36.csv

In [149]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 36

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

tipo_cadena


,tipo_cadena,valor
0,Miel,0.86
1,Té,0.85
2,Sorgo,0.76
3,Papa,0.73
4,Olivo,0.71


In [150]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,tipo_cadena,indicador,valor
0,Ajo,valor,0.11
1,Algodón,valor,0.48
2,Arroz,valor,0.12
3,Avicola,valor,0.06
4,Berries,valor,0.02


In [151]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [152]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g36.csv',
 '../output/AGROPE1/definitivos/AGROPE_g36_old.csv')

AGROPE_g37.csv

In [153]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 37

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio


,anio,productos_primarios,moa,participacion_expo_totales
0,2009,8124.0,21212.0,0.526943
1,2010,13637.0,22661.0,0.532428
2,2011,18373.0,28192.0,0.561152
3,2012,16934.0,27474.0,0.555222
4,2013,17258.0,28938.0,0.608138


In [154]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="valor"),
    sort(primary_keys),
    # renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,indicador,valor
0,2009,productos_primarios,8124.000000
1,2009,moa,21212.000000
2,2009,participacion_expo_totales,0.526943
3,2010,moa,22661.000000
4,2010,participacion_expo_totales,0.532428


In [155]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [156]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g37.csv',
 '../output/AGROPE1/definitivos/AGROPE_g37_old.csv')

AGROPE_g38.csv

In [157]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
grafico_n = 38

archivo = f'{SUBTOP}_g{grafico_n}'
df = pd.read_csv(f"{folder}/definitivos/{archivo}.csv")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print(*primary_keys)
df.head()

anio producto_exportacion


,anio,producto_exportacion,valor
0,1986,cereales,77.9
1,1987,cereales,65.2
2,1988,cereales,78.0
3,1989,cereales,109.3
4,1990,cereales,106.4


In [158]:
drop = drop_colx
to_long = wide_to_longx
renombrar = rename_colsx
nomenclar_paises = partial(normalize_countriesx, map=pais)
replace = replace_valuex
sort = sort_vals_ascx
cast_column = cast_colx
exportar = lambda df: exportar_definitivo(archivo=archivo, folder=folder, df=df, nuevo_nombre=None)



pipeline = chain(
    # nomenclar_paises(code_col='iso3', new_col=None, desc_col='iso3_desc_fundar'),
    # drop(col="countryname"),
    # cast_column(col="anio", cast_to="int"),
    to_long(primary_keys , value_name="_valor"),
    sort(primary_keys),
    renombrar({'_valor':'valor'}),
    # replace(col='pov_type', curr_value='with_transfers', new_value='Con transferencias'),
    # replace(col='pov_type', curr_value='without_transfers', new_value='Sin transferencias'),
    # replace(col='pov_type', curr_value='difference', new_value='Diferencia')
    # drop('region_name'), 
    # renombrar({'time': 'anio'}),
    # nomenclar_paises(code_col='cod_area', new_col="desc_area"), 
    #renombrar({'iso3': 'pais'}),
   
)

_df = pipeline(df)

diccionario_cambios[archivo] = lista_cambios.copy()

_df.head()

,anio,producto_exportacion,indicador,valor
0,1986,carnes_preparados,valor,86.9
1,1986,cereales,valor,77.9
2,1986,grasas_aceites,valor,57.4
3,1986,residuos_alimenticios,valor,89.7
4,1986,semillas_frutos_oleaginosos,valor,71.9


In [159]:
diccionario_cambios 

{'AGROPE_g1': ['Long estricto: [anio, participacion_agro] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g2': ['Long estricto: [anio, pbi_agro_pcons] -> [anio, valor, indicador]',
  'Ordené las filas por las columnas: anio'],
 'AGROPE_g3': ['Long estricto: [anio, cuenta, participacion_pbi] -> [anio, cuenta, valor, indicador]',
  'Ordené las filas por las columnas: anio, cuenta'],
 'AGROPE_g4': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, anio, va_agro_sobre_pbi, pais] -> [iso3c, anio, pais, valor, indicador]',
  'Ordené las filas por las columnas: iso3c, anio'],
 'AGROPE_g5': ['Usando los valores de la columna iso3c, se normalizaron los nombres de las regiones/paises con el nomenclador vigente al 2024-03-15, cambiando los valores de la columna pais',
  'Long estricto: [iso3c, pais, anio, va_a

In [160]:
exportar(_df)

('../output/AGROPE1/definitivos/AGROPE_g38.csv',
 '../output/AGROPE1/definitivos/AGROPE_g38_old.csv')

In [161]:
SUBTOP = 'AGROPE'
entrega = 1
folder = f"../output/{SUBTOP}{entrega}"
match_relocate(f'{folder}/definitivos', f'{folder}/old', '.*old.*')

Files matching the pattern 're.compile('.*old.*')' moved to '../output/AGROPE1/old'.


In [162]:
import json
import datetime

today = datetime.datetime.now().strftime("%Y-%m-%d")

with open(f'../output/AGROPE1/definitivos/paso_a_definitivos_{today}.json', 'w') as fp:
    json.dump(diccionario_cambios, fp, indent=4, ensure_ascii=False)
